# Electric Scooter - Project

**Team Members - MSBA '24 Morning Cohort:**
- Karthick Vel Kathirvel (kk37347)
- Sharan Arora (sa44827)
- Spandan Pal (sp54969)
- Manideep Telukuntla (mt39528)
- Pritesh Singh (ps35762)
- Raghav Vaidya (rv25256)


In [ ]:
#pip install --upgrade google-api-python-client

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/f8/e9/11fb73fe17f63e2e8aa7c1f4afb790ab737c0881dce4757bdead998ef48d/google_api_python_client-2.102.0-py2.py3-none-any.whl.metadata
  Using cached google_api_python_client-2.102.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Obtaining dependency information for google-auth<3.0.0.dev0,>=1.19.0 from https://files.pythonhosted.org/packages/d7/88/1826b0c047c48763b36ed854a984127b430a16b70003155d7b19975f1d59/google_auth-2.23.2-py2.py3-none-any.whl.metadata
  Using cached google_auth-2.23.2-py2.py3-none-any.whl.metadata (4.2 kB)
  Obtaining dependency information for google-auth-httplib2>=0.1.0 from https://files.pythonhosted.org/packages/d3/3d/e4991229886c0d522d9552151a43ff7adcc61e026e60ce8bd508387f84cf/google_auth_httplib2-0.1.1-py2.py3-none-any.whl.metadata
  Using cached google_auth_httplib2-0.1.1-py2.py3-none-any.whl.metadata (2.1 

In [ ]:
import pandas as pd
from googleapiclient.discovery import build

In [ ]:
# Example DataFrame
products = ['Wheelspeed Primer', 'Gotrax GXL V2', 'VOLPAM SPT7']

In [ ]:
#read file using with and read a string
with open('youtubeKey.txt', 'r') as f:
    youtubeKey = f.readline()

In [ ]:
youtube = build('youtube', 'v3', developerKey=youtubeKey)

In [ ]:
def search_videos(youtube, query):
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        maxResults=10,
        type="video"
    )
    response = request.execute()
    return response['items']

In [ ]:
for i in range(len(products)):
    items = search_videos(youtube, products[i])
    titles = [item['snippet']['title'] for item in items]
    urls = [f"https://www.youtube.com/watch?v={item['id']['videoId']}" for item in items]
    descriptions = [item['snippet']['description'] for item in items]
    temp_df = pd.DataFrame({'Product': products[i], 'Video_Title': titles, 'Video_URL': urls, 'Video_Description': descriptions})
    if i == 0:
        product1 = temp_df.copy()
    if i == 1:
        product2 = temp_df.copy()
    if i == 2:
        product3 = temp_df.copy()

In [ ]:
#read file using with and read a string
with open('openAIKey.txt', 'r') as f:
    openAIKey = f.readline()

In [ ]:
prompts = []
for i in range(3):
    if i == 0:
        product = product1.copy()
    if i == 1:
        product = product2.copy()
    if i == 2:
        product = product3.copy()
    prompt = "Product I am looking for: " + product['Product'][0] + "\n" + "Are the video titles below strictly about the product I am looking for? Answer yes or no for each title with the same format as below." + "\n" + "Video Titles: " + '\n'.join(product.index.astype("str") + ': ' + product['Video_Title'])
    prompts.append(prompt)

In [ ]:
import openai
openai.api_key = openAIKey

In [ ]:
def queryGPT(queryPrompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
            "role": "user",
            "content": queryPrompt
            }
        ],
        temperature=1,
        max_tokens=3000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response

In [ ]:
prod1VideosToLookFor = [int(i.split(': ')[0]) for i in vars(queryGPT(prompts[0]))['_previous']['choices'][0]['message']['content'].split('\n') if i.split(': ')[1] == 'Yes']
prod2VideosToLookFor = [int(i.split(': ')[0]) for i in vars(queryGPT(prompts[1]))['_previous']['choices'][0]['message']['content'].split('\n') if i.split(': ')[1] == 'Yes']
prod3VideosToLookFor = [int(i.split(': ')[0]) for i in vars(queryGPT(prompts[2]))['_previous']['choices'][0]['message']['content'].split('\n') if i.split(': ')[1] == 'Yes']

In [ ]:
prod1VideosToLookFor = product1.iloc[prod1VideosToLookFor]
prod2VideosToLookFor = product2.iloc[prod2VideosToLookFor]
prod3VideosToLookFor = product3.iloc[prod3VideosToLookFor]

In [ ]:
prod1VideosToLookFor['videoID'] = prod1VideosToLookFor['Video_URL'].str.split('v=').str[1]
prod2VideosToLookFor['videoID'] = prod2VideosToLookFor['Video_URL'].str.split('v=').str[1]
prod3VideosToLookFor['videoID'] = prod3VideosToLookFor['Video_URL'].str.split('v=').str[1]

C:\Users\kkart\AppData\Local\Temp\ipykernel_21260\99434895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod1VideosToLookFor['videoID'] = prod1VideosToLookFor['Video_URL'].str.split('v=').str[1]
C:\Users\kkart\AppData\Local\Temp\ipykernel_21260\99434895.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod3VideosToLookFor['videoID'] = prod3VideosToLookFor['Video_URL'].str.split('v=').str[1]


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
def fetch_and_analyze(row):
    video_id = row['videoID']
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
    except Exception as e:
        print(f"An error occurred for video {video_id}: {e}")
        return None  # Return None for missing transcripts

    text = ''
    for entry in transcript:
        text += ' ' + entry['text']

    newPrompt = "Find pros and cons of the product below:" + "\n" + text

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {
                    "role": "user",
                    "content": newPrompt
                }
            ],
            temperature=1,
            max_tokens=3000,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
    except Exception as e:
        print(f"An error occurred while generating response for video {video_id}: {e}")
        return None  # Return None for failed API calls

    reviewResponse = vars(response)['_previous']['choices'][0]['message']['content']
    return reviewResponse

prod1VideosToLookFor['reviewResponse'] = prod1VideosToLookFor.apply(fetch_and_analyze, axis=1)
prod2VideosToLookFor['reviewResponse'] = prod2VideosToLookFor.apply(fetch_and_analyze, axis=1)
prod3VideosToLookFor['reviewResponse'] = prod3VideosToLookFor.apply(fetch_and_analyze, axis=1)


An error occurred for video ptT_TREML5g: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=ptT_TREML5g! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!


C:\Users\kkart\AppData\Local\Temp\ipykernel_21260\2852052447.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod1VideosToLookFor['reviewResponse'] = prod1VideosToLookFor.apply(fetch_and_analyze, axis=1)
C:\Users\kkart\AppData\Local\Temp\ipykernel_21260\2852052447.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod3VideosToLookFor['reviewResponse'] = prod3VideosToLookFor.apply(fetch_and_analyze, axis=1)


In [ ]:
prod1VideosToLookFor = prod1VideosToLookFor[~prod1VideosToLookFor['reviewResponse'].isnull()]
prod2VideosToLookFor = prod2VideosToLookFor[~prod2VideosToLookFor['reviewResponse'].isnull()]
prod3VideosToLookFor = prod3VideosToLookFor[~prod3VideosToLookFor['reviewResponse'].isnull()]

In [ ]:
#use reduce to combine all the reviews into one string
from functools import reduce
summaryWithinProduct1Prompt = "Remove recurring pros and cons from multiple reviews separated by 'NextReview' and collate a final version of pros and cons from the following: \n" + reduce(lambda x, y: x + '\n NewReview \n' + y, prod1VideosToLookFor['reviewResponse']) + "\n" + "Somtimes the entire review can be irrelevant to scooter. Do not consider them for pros and cons. Only return keywords in each pro and con and keep it concise. Limit to 5 pros and 5 cons."
summaryWithinProduct2Prompt = "Remove recurring pros and cons from multiple reviews separated by 'NextReview' and collate a final version of pros and cons from the following: \n" + reduce(lambda x, y: x + '\n NewReview \n' + y, prod2VideosToLookFor['reviewResponse']) + "\n" + "Somtimes the entire review can be irrelevant to scooter. Do not consider them for pros and cons. Only return keywords in each pro and con and keep it concise. Limit to 5 pros and 5 cons."
summaryWithinProduct3Prompt = "Remove recurring pros and cons from multiple reviews separated by 'NextReview' and collate a final version of pros and cons from the following: \n" + reduce(lambda x, y: x + '\n NewReview \n' + y, prod3VideosToLookFor['reviewResponse']) + "\n" + "Somtimes the entire review can be irrelevant to scooter. Do not consider them for pros and cons. Only return keywords in each pro and con and keep it concise. Limit to 5 pros and 5 cons."

In [ ]:
#use GPT to summarize the reviews
finalReviewWithinProduct1 = vars(queryGPT(summaryWithinProduct1Prompt))['_previous']['choices'][0]['message']['content']
finalReviewWithinProduct2 = vars(queryGPT(summaryWithinProduct2Prompt))['_previous']['choices'][0]['message']['content']
finalReviewWithinProduct3 = vars(queryGPT(summaryWithinProduct3Prompt))['_previous']['choices'][0]['message']['content']

In [ ]:
finalReviewWithinProduct1.split('\n')

['Pros:',
 '1. User-friendly with easy-to-follow instructions.',
 '2. Features speed control modes.',
 '3. Password protection system for safety.',
 '4. Cruise control functionality.',
 '5. Reliable brakes and built-in horn.',
 '',
 'Cons:',
 '1. Confusing product packaging.',
 '2. Top speed of 15 mph may be insufficient.',
 '3. Password entry required each time it is turned on.',
 '4. Motor locks after unsuccessful password entry.',
 '5. No option to disable light in pedestrian mode.']

In [ ]:
finalReviewWithinProduct2.split('\n')

['Final Collated Pros and Cons: ',
 '',
 'Pros:',
 '1. Easy Assembly: Simple and straightforward setup and control system.  ',
 '2. Performance: Offers respectable speed and range; performs well on different terrains. ',
 '3. Features: Comes with an efficient brake system, has a LED headlight, and a display for speed and battery gauge.',
 '4. Long-Lasting: Good battery longevity and overall scooter durability. ',
 '5. Portability: Easy to fold and transport thanks to its lightweight design. ',
 '',
 'Cons:',
 '1. Fender Lock: Factory settings need adjusting for optimal functionality.',
 '2. Battery Issues: Degraded battery performance over time; unreliable battery gauge.',
 '3. Consistency: Potential inconsistency in top speed between scooters of the same model.',
 '4. Lights: Headlight not sufficiently bright; lacks brake light for safety during night-time rides.',
 '5. Maintenance: Regular maintenance is needed, particularly for tires and brakes; degradation in scooter range observed

In [ ]:
finalReviewWithinProduct3.split('\n')

['Final Revised Review: ',
 '',
 'Pros:',
 '1. Exact product as ordered, satisfying customer expectations.',
 '2. Dual shocks at front and rear for a smoother ride.',
 '3. Simple assembly process with a few screws to tighten.',
 '4. Comes fully charged for immediate test ride.',
 '5. Features turn signal, brake light, headlamp and rear light for safety and visibility during night rides.',
 '',
 'Cons:',
 '1. Height adjustment can be challenging initially.',
 '2. Adjustment screw required for closing and locking might be confusing.',
 '3. Activation of headlamp involves a certain sequence of button pressing which can be confusing. ',
 '4. Setup process requires patience and understanding of instructions.']